# 환경 세팅

In [9]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 120)
pd.options.display.float_format = '{:,.2f}'.format

# 다대다 병합
* 다대다 병합 : 병합 기준 값이 왼쪽, 오른쪽 모두에서 중복일 때 사용하는 병합  
1) 그러나, 위와 같은 경우여도 '다대다 병합'을 해야하는 경우는 드묾 / 그 대신 오히려 '일대다 관계'를 복원하는 게 우선일 수가 있음
2) 다대다 병합이 불가피한 경우도 있음 => 중복이 발생하는 것에 상관없이 전체 데이터를 포함해야하는 경우.. 

In [2]:
import pandas as pd
import os
print(os.listdir('data'))

['cmacitations.csv', 'cmacitations.pkl', 'cmacreators.csv', 'cmacreators.pkl', 'ghcn', 'ltcountries.csv', 'ltcountries.pkl', 'ltcountry', 'ltlocations.csv', 'ltlocations.pkl', 'nls97add.csv', 'nls97add.pkl', 'nls97colenr.csv', 'nls97colenr.pkl', 'nls97f.csv', 'nls97f.pkl', 'nls97weeksworked.csv', 'nls97weeksworked.pkl']


In [4]:
cmacitations = pd.read_csv('data/cmacitations.csv') # 클리블랜드 미술관 소장품별 매체 인용 
cmacreators = pd.read_csv('data/cmacreators.csv') # 클리블랜드 미술관 소장품별 작가 정보

## 데이터 살펴보기

In [6]:
cmacitations.head(10)

,id,citation
0,92937,"Milliken, William M. ""The Second Exhibition of..."
1,92937,"Glasier, Jessie C. ""Museum Gets Prize-Winning ..."
2,92937,"""Cleveland Museum Acquires Typical Pictures by..."
3,92937,"Milliken, William M. ""Two Examples of Modern P..."
4,92937,<em>Memorial Exhibition of the Work of George ...
5,92937,The Cleveland Museum of Art. <em>Handbook of t...
6,92937,"Cortissoz, Royal. ""Paintings and Prints by Geo..."
7,92937,"Isham, Samuel, and Royal Cortissoz. <em>The Hi..."
8,92937,"Mather, Frank Jewett, Charles Rufus Morey, and..."
9,92937,"""Un Artiste Americain."" <em>L'illustration.</e..."


In [7]:
cmacitations.shape

(11642, 2)

In [8]:
cmacitations.id.nunique()

935

In [11]:
cmacreators.columns

Index(['id', 'creator', 'title', 'birth_year', 'death_year', 'collection',
       'type', 'creation_date'],
      dtype='object')

In [13]:
cmacreators.loc[:,['id','creator','birth_year']].head(10)

,id,creator,birth_year
0,92937,"George Bellows (American, 1882-1925)",1882
1,94979,"John Singleton Copley (American, 1738-1815)",1738
2,137259,"Gustave Courbet (French, 1819-1877)",1819
3,141639,"Frederic Edwin Church (American, 1826-1900)",1826
4,93014,"Thomas Cole (American, 1801-1848)",1801
5,110180,"Albert Pinkham Ryder (American, 1847-1917)",1847
6,135299,"Vincent van Gogh (Dutch, 1853-1890)",1853
7,125249,"Vincent van Gogh (Dutch, 1853-1890)",1853
8,126769,"Henri Rousseau (French, 1844-1910)",1844
9,135382,"Claude Monet (French, 1840-1926)",1840


In [17]:
cmacitations.id.value_counts().head(10)
##### citations 데이터에서 '병합 기준 값'의  중복을 보임 - 예. 소장품 148758에 대해 174회의 인용이 발생

148758    174
122351    116
92937      98
123168     94
149112     93
94979      93
124245     87
128842     86
102578     84
93014      79
Name: id, dtype: int64

In [20]:
cmacreators.id.value_counts().head(10)
# -> creators 데이터에서도 '병합 기준 값'이 중복을 보임 - 예. 소장품 149386에 대해 4번의 작가 정보가 발생

149386    4
140001    4
140427    3
146797    3
93173     3
146795    3
142752    3
114537    3
149041    3
114538    3
Name: id, dtype: int64

## 병합 확인

In [28]:
### dfleft  : 왼쪽 데이터프레임 
### dfright : 오른쪽 데이터프레임
### idvar   : 병합 기준 열
def checkmerge(dfleft, dfright, idvar):
    dfleft['inleft'] = 'Y'
    dfright['inright'] = 'Y'
    dfboth = pd.merge(dfleft[[idvar, 'inleft']], dfright[[idvar, 'inright']],\
                      on=[idvar], how='outer')
    dfboth.fillna('N', inplace=True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))

checkmerge(cmacitations, cmacreators, 'id')

inright     N     Y
inleft             
N           0    46
Y        2579  9701


## 양쪽 중복 값 보이기

In [30]:
cmacitations.loc[cmacitations.id==124733,:] ### 특정 소장품에 대한 14회의 인용

,id,citation,inleft
8963,124733,"Weigel, J. A. G. <em>Catalog einer Sammlung vo...",Y
8964,124733,"Winkler, Friedrich. <em>Die Zeichnungen Albrec...",Y
8965,124733,"Francis, Henry S. ""Drawing of a Dead Blue Jay ...",Y
8966,124733,"Kurz, Otto. <em>Fakes: A Handbook for Collecto...",Y
8967,124733,Minneapolis Institute of Arts. <em>Watercolors...,Y
8968,124733,"Pilz, Kurt. ""Hans Hoffmann: Ein Nürnberger Dür...",Y
8969,124733,"Koschatzky, Walter and Alice Strobl. <em>Düre...",Y
8970,124733,"Johnson, Mark M<em>. Idea to Image: Preparator...",Y
8971,124733,"Kaufmann, Thomas DaCosta. <em>Drawings from th...",Y
8972,124733,"Koreny, Fritz. <em>Albrecht Dürer and the ani...",Y


In [31]:
cmacreators.loc[cmacreators.id==124733, ['id','creator','birth_year','title']] ### 특정 소장품에 대한 2명의 작가

,id,creator,birth_year,title
449,124733,"Albrecht Dürer (German, 1471-1528)",1471,Dead Blue Roller
450,124733,"Hans Hoffmann (German, 1545/50-1591/92)",1545/50,Dead Blue Roller


## 다대다 병합 실행

In [36]:
cma = pd.merge(cmacitations, cmacreators, on=['id'], how='outer')
cma['citation'] = cma.citation.str[0:20]
cma['creator'] = cma.creator.str[0:20]
cma.loc[cma.id==124733, ['citation','creator','birth_year']].shape ### 특정 소장품에 대한 작가별 인용이 총 28회

,citation,creator,birth_year
9457,"Weigel, J. A. G. <em",Albrecht Dürer (Germ,1471
9458,"Weigel, J. A. G. <em",Hans Hoffmann (Germa,1545/50
9459,"Winkler, Friedrich.",Albrecht Dürer (Germ,1471
9460,"Winkler, Friedrich.",Hans Hoffmann (Germa,1545/50
9461,"Francis, Henry S. ""D",Albrecht Dürer (Germ,1471
9462,"Francis, Henry S. ""D",Hans Hoffmann (Germa,1545/50
9463,"Kurz, Otto. <em>Fake",Albrecht Dürer (Germ,1471
9464,"Kurz, Otto. <em>Fake",Hans Hoffmann (Germa,1545/50
9465,Minneapolis Institut,Albrecht Dürer (Germ,1471
9466,Minneapolis Institut,Hans Hoffmann (Germa,1545/50


In [37]:
cma.loc[cma.id==124733, ['citation','creator','birth_year']].shape

(28, 3)

In [39]:
cmacreators.creator.nunique()

475

# 병합 루틴
* 데이터 병합 전후의 손상을 피하는 일반적인 방법을 택하면(테크닉), 더 중요한 것(데이터 자체)에 초점을 맞출 수 있다.
* 병합 루틴 
        (1) 병합 기준 값이 각 데이터에서 고유한지 확인 
        (2) 병합 기준 값 불일치를 먼저 확인
        (3) 조인 수행
        (4) 결과 확인
* 병합 루틴 실습 과제 : GHCN 데이터베이스의 countries 데이터를 locations 데이터와 왼쪽 조인

## 데이터 로딩 

In [8]:
import pandas as pd
countries = pd.read_csv('data/ltcountries.csv')
locations = pd.read_csv('data/ltlocations.csv')

## 병합 기준 값이 각 데이터에서 고유한지 확인

In [9]:
print(countries.countryid.nunique()==countries.shape[0])
print(locations.countryid.nunique()==locations.shape[0])

True
False


## 병합 기준 열 일치/불일치 여부 확인 ❕
- dfleft = 왼쪽 데이터 
- dfright = 오른쪽 데이터 
- mergebyleft = 왼쪽 데이터의 병합 기준 열
- mergebyright = 오른쪽 데이터의 병합 기준 열

In [12]:
def checkmerge(dfleft, dfright, mergebyleft, mergebyright):
    dfleft['in_left'] = 'Y'
    dfright['in_right'] = 'Y'
    dfboth = pd.merge(dfleft[[mergebyleft, 'in_left']],
                     dfright[[mergebyright, 'in_right']],
                     left_on=[mergebyleft],
                     right_on=[mergebyright],
                     how='outer')
    dfboth.fillna('N', inplace=True)
    print(pd.crosstab(dfboth.in_left, dfboth.in_right))
    print(dfboth.loc[(dfboth.in_left=='N')|(dfboth.in_right=='N')].head(20))
    
checkmerge(countries.copy(), locations.copy(), 'countryid', 'countryid')

in_right  N      Y
in_left           
N         0      1
Y         2  27472
      countryid in_left in_right
9715         LQ       Y        N
13103        ST       Y        N
27474        FO       N        Y


## 조인 수행

In [ ]:
stations = pd.merge(countries, locations,
                   left_on=['countryid'],
                   right_on=['countryid'],
                   how='left')
print(stations.shape)
stations[['locationid','latitude','stnelev','country']].head(10)

## 병합에 관한 Tip
    1. checkmerge 함수를 매번 재작성할 필요 없이 "모듈"에 넣어두고 import할 수 있다.
        (chapter10의 헬퍼 함수 추가 방법 학습예정)
    2. left_on, right_on 인자를 사용하여 명시적으로 병합 기준 열 지정해서 사용
    3. how 매개변수의 기본값으로 "왼쪽 조인" 사용하기
        >> 왼쪽 : 분석 단위(학생, 환자, 고객, activity 등..)
        >> 오른쪽 : 보충 데이터(성정, 혈압, 우편번호, resource 등..)
      cf1. 그 외의 조인을 할 이유가 있는지를 생각해보는 식으로 정해두기~
      cf2. 보충 데이터에 나타나지 않는 병합 기준 값 있는데 내부 조인을 하면 분석 단위 행 사라지는 문제